In [ ]:
import openpyxl
import pandas as pd
import os
from openpyxl.styles import Border, Side, Alignment, PatternFill, Font
import jpype

from asposecells.api import Workbook, PdfSaveOptions, SaveFormat


from asposecells.api import Workbook, FileFormatType, PdfSaveOptions

# Load the Excel file
wb = openpyxl.load_workbook(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\BR-CG-CEIS_02262024.xlsx")
# Get the file name from the path
filename_excel = os.path.splitext(os.path.basename(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\BR-CG-CEIS_02262024.xlsx"))[0]
print(filename_excel)
# Read the source workbook into a pandas DataFrame
df = pd.read_excel(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\CITATION_CONFIG.xlsx", sheet_name='Sheet1')

# Check if the search title exists in the specified column
if filename_excel in df['RPL_TYPE'].values:
    # Get the row(s) where the search title is found
    matching_rows = df[df['RPL_TYPE'] == filename_excel]

    # Retrieve the corresponding adjacent row values
    adjacent_row_values = matching_rows.values.tolist()[0]

    print(adjacent_row_values)
else:
    print("Search Title not found in the specified column.")

# Get the worksheet
ws = wb.active

#  Wrap the text in the selected cells and set the border
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.alignment = Alignment(vertical='top', horizontal='left', wrap_text=True) # Set alignment to top-left
        cell.border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
        ws.row_dimensions[cell.row].height = max(14, int(cell.font.size) + 2)


        # if len(str(cell.value)) > 20:  # Modify the threshold as needed
        #     cell.alignment = cell.alignment.copy(wrapText=True)
        
     

        

# Set the worksheet orientation to landscape
ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE

# Color the header row with yellow background
header_fill = PatternFill(fill_type="solid", fgColor="FFFF00")
for cell in ws[1]:
    cell.fill = header_fill

# Save the Excel file
wb.save('output.xlsx')


workbook = Workbook("output.xlsx")
for sheet in workbook.getWorksheets():
   sheet.autoFitColumns()
   sheet.autoFitRows()
  

# Get the default worksheet
worksheet = workbook.getWorksheets().get(0)

# Set the page orientation to landscape
worksheet.getPageSetup().setOrientation(2)

# Set the paper size to A4

worksheet.getPageSetup().setPaperSize(9)
# Set the maximum rows per page
worksheet.getPageSetup().setFitToPagesTall(1)
worksheet.getPageSetup().setFitToPagesWide(0)
worksheet.setPageBreakPreview(False)



# Set the vertical zoom level to fit 1000 rows per page
# worksheet.setZoom(1000)

column_indices = [4, 5, 6, 9, 21, 22, 13, 14, 17]
column_width = 35

for index in column_indices:
    sheet.getCells().setColumnWidth(index, column_width)
cells = sheet.getCells()
for row in cells.getRows():
    for index in column_indices:
        cell = row.get(index)
        style = cell.getStyle()
        style.setTextWrapped(True)
        cell.setStyle(style)
        
# for sheet in workbook.getWorksheets():
#    sheet.autoFitColumns()
#    sheet.autoFitRows()

# Save as PDF
pdf_options = PdfSaveOptions()
pdf_options.setOnePagePerSheet(True)
# pdf_options.setAllColumnsInOnePagePerSheet(True)
workbook.save("output.pdf", pdf_options)

                     ##################################TEST CODE######################################################

In [134]:
import json
import openpyxl
import pandas as pd
from openpyxl.styles import Border, Side, Alignment, PatternFill
from asposecells.api import Workbook, PdfSaveOptions
def load_excel(file_path):
   return openpyxl.load_workbook(file_path)
def read_dataframe(file_path):
   return pd.read_excel(file_path)
def get_adjacent_row_values(df_config, search_title):
   matching_rows = df_config[df_config['RPL_TYPE'] == search_title]
   if not matching_rows.empty:
       adjacent_row_values = json.loads(matching_rows.iloc[0, 1])
       return adjacent_row_values
   else:
       print("Search Title not found in the specified column.")
       return {}
def get_column_indices(df, adjacent_row_values):
   column_indices = {}
   for column_name, value in adjacent_row_values.items():
       if column_name in df.columns:
           column_index = df.columns.get_loc(column_name)
           column_indices[column_index] = value
   return column_indices
def apply_formatting_to_worksheet(ws, column_indices):
    for row in ws.iter_rows(min_row=1, min_col=1, max_row=ws.max_row, max_col=ws.max_column):
        for cell in row:
            cell.alignment = Alignment(vertical='top', horizontal='left', wrap_text=True)
            cell.border = Border(left=Side(style='thin'), right=Side(style='thin'),
                                 top=Side(style='thin'), bottom=Side(style='thin'))
            ws.row_dimensions[cell.row].height = max(14, int(cell.font.size) + 2)

    yellow_fill = PatternFill(fill_type="solid", fgColor="FFFF00")
    for cell in ws[1]:
        cell.fill = yellow_fill
def save_workbook_as_pdf(workbook, output_pdf_file_path):
   pdf_options = PdfSaveOptions()
   pdf_options.setOnePagePerSheet(True)
   workbook.save(output_pdf_file_path, pdf_options)
def main():
   excel_file_path = r"BR-CG-CEIS_02262024.xlsx"
   config_file_path = r"CITATION_CONFIG.xlsx"
   output_pdf_file_path = "output.pdf"
   sheet_name = 'Sheet1'
   wb = load_excel(excel_file_path)
   df = read_dataframe(excel_file_path)
   df_config = read_dataframe(config_file_path)
   filename_excel = os.path.splitext(os.path.basename(excel_file_path))[0]
   adjacent_row_values = get_adjacent_row_values(df_config, filename_excel)
   column_indices = get_column_indices(df, adjacent_row_values)
   sheet = wb[sheet_name]
   apply_formatting_to_worksheet(sheet, column_indices)
   wb.save('output.xlsx')
   output_wb = Workbook("output.xlsx")
   for sheet in output_wb.getWorksheets():
       sheet.autoFitColumns()
       sheet.autoFitRows()
   worksheet = output_wb.getWorksheets().get(0)
   worksheet.getPageSetup().setOrientation(2)
   worksheet.getPageSetup().setPaperSize(9)
   worksheet.getPageSetup().setFitToPagesTall(1)
   worksheet.getPageSetup().setFitToPagesWide(0)
   worksheet.setPageBreakPreview(False)
   for index, width in column_indices.items():
       sheet.getCells().setColumnWidth(index, width)
   cells = sheet.getCells()
   for row in cells.getRows():
       for index in column_indices:
           cell = row.get(index)
           style = cell.getStyle()
           style.setTextWrapped(True)
           cell.setStyle(style)
   save_workbook_as_pdf(output_wb, output_pdf_file_path)
if __name__ == "__main__":
   main()

In [ ]:
import json
import pandas as pd
import streamlit as st
from openpyxl.styles import Border, Side, Alignment, PatternFill
from asposecells.api import Workbook, PdfSaveOptions
def process_excel_to_pdf(excel_file_path, config_file_path):
   # Load Excel file and read config
   wb = Workbook(excel_file_path)
   df = pd.read_excel(excel_file_path)
   df_config = pd.read_excel(config_file_path)
   # Get filename and adjacent row values
   filename_excel = os.path.splitext(os.path.basename(excel_file_path))[0]
   adjacent_row_values = get_adjacent_row_values(df_config, filename_excel)
   column_indices = get_column_indices(df, adjacent_row_values)
   # Apply formatting to worksheet
   sheet = wb.getWorksheets().get(0)
   apply_formatting_to_worksheet(sheet, column_indices)
   # Save Excel file
   output_excel_file_path = "output.xlsx"
   wb.save(output_excel_file_path)
   # Save PDF
   output_pdf_file_path = "output.pdf"
   output_wb = Workbook(output_excel_file_path)
   for sheet in output_wb.getWorksheets():
       sheet.autoFitColumns()
       sheet.autoFitRows()
   worksheet = output_wb.getWorksheets().get(0)
   worksheet.getPageSetup().setOrientation(2)
   worksheet.getPageSetup().setPaperSize(9)
   worksheet.getPageSetup().setFitToPagesTall(1)
   worksheet.getPageSetup().setFitToPagesWide(0)
   worksheet.setPageBreakPreview(False)
   for index, width in column_indices.items():
       sheet.getCells().setColumnWidth(index, width)
   cells = sheet.getCells()
   for row in cells.getRows():
       for index in column_indices:
           cell = row.get(index)
           style = cell.getStyle()
           style.setTextWrapped(True)
           cell.setStyle(style)
   pdf_options = PdfSaveOptions()
   pdf_options.setOnePagePerSheet(True)
   output_wb.save(output_pdf_file_path, pdf_options)
   return output_pdf_file_path
def get_adjacent_row_values(df_config, search_title):
   matching_rows = df_config[df_config['RPL_TYPE'] == search_title]
   if not matching_rows.empty:
       adjacent_row_values = json.loads(matching_rows.iloc[0, 1])
       return adjacent_row_values
   else:
       st.error("Search Title not found in the specified column.")
def get_column_indices(df, adjacent_row_values):
   column_indices = {}
   for column_name, value in adjacent_row_values.items():
       if column_name in df.columns:
           column_index = df.columns.get_loc(column_name)
           column_indices[column_index] = value
   return column_indices
def apply_formatting_to_worksheet(ws, column_indices):
   for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
       for cell in row:
           cell.alignment = Alignment(vertical='top', horizontal='left', wrap_text=True)
           cell.border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
           ws.row_dimensions[cell.row].height = max(14, int(cell.font.size) + 2)
   ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
   header_fill = PatternFill(fill_type="solid", fgColor="FFFF00")
   for cell in ws[1]:
       cell.fill = header_fill
st.title("Excel to PDF Converter")
excel_file = st.file_uploader("Upload Excel file", type=["xlsx"])
config_file = st.file_uploader("Upload Config file", type=["xlsx"])
if excel_file and config_file:
   pdf_file_path = process_excel_to_pdf(excel_file, config_file)
   st.success("PDF successfully generated!")
   st.markdown(f"Download [PDF file](/{pdf_file_path})")
   if st.button("Download PDF"):
       st.download_button(label="Download PDF", data=open(pdf_file_path, "rb").read(), file_name="output.pdf", mime="application/pdf")

In [137]:
import requests

url = "http://iacap013.dev.e2open.com:9080/feed/v1/feeds"

# Request parameters
params = {
    "source": "Feed Management",
    "tid": "LOGITECH",
    "partnerId": "11445",
    "fileName": "testing.xlsx",
    "partnerDomainName": "11445",
    "comments": "",
    "feedCategory": "",
    "emailId": "pratap.chitipothu@e2open.c",
    "userName": "Pratap Kumar Chitipot",
    "userType": "customer",
    "resubmissionReason": ""
}

# File path to upload
file_path = r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\CITATION_CONFIG.xlsx"

# Open the file in binary mode
with open(file_path, "rb") as file:
    # Create the multipart form data payload
    payload = {
        "file": file
    }

    # Send the POST request with the file
    response = requests.post(url, params=params, files=payload)

# Check the response
if response.status_code == 200:
    print("File upload successful")
else:
    print("File upload failed")

File upload successful


In [135]:
import json
import openpyxl
import pandas as pd
import os
from openpyxl.styles import Border, Side, Alignment, PatternFill, Font
import jpype

from asposecells.api import Workbook, PdfSaveOptions, SaveFormat

 
# Load the Excel file
wb = openpyxl.load_workbook(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\BR-CG-CEIS_02262024.xlsx")
df = pd.read_excel(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\BR-CG-CEIS_02262024.xlsx")
 
 
# Get the file name from the path
filename_excel = os.path.splitext(os.path.basename(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\BR-CG-CEIS_02262024.xlsx"))[0]
# print(filename_excel)
# Read the source workbook into a pandas DataFrame
df_config = pd.read_excel(r"C:\Users\PRKUMAR\OneDrive - e2open, LLC\PycharmProjects\PDFCitation\CITATION_CONFIG.xlsx", sheet_name='Sheet1')
 
# Check if the search title exists in the specified column
if filename_excel in df_config['RPL_TYPE'].values:
    # Get the row(s) where the search title is found
    matching_rows = df_config[df_config['RPL_TYPE'] == filename_excel]
 
    # Retrieve the corresponding adjacent row values
 
    adjacent_row_values = matching_rows.iloc[0, 1]
    adjacent_row_values = json.loads(adjacent_row_values)
    print(adjacent_row_values)
else:
    print("Search Title not found in the specified column.")
 
 
column_names_dict = {'NOME DO SANCIONADO': 35, 'NOME INFORMADO PELO ÓRGÃO SANCIONADOR':35, 'RAZÃO SOCIAL - CADASTRO RECEITA': 35, 'CATEGORIA DA SANÇÃO': 35, 'ÓRGÃO SANCIONADOR': 35, "DETALHAMENTO":10}
# Dictionary to store column indices
column_indices = {}
# Iterate over the dictionary and check if the column exists in the DataFrame
for column_name, value in adjacent_row_values.items():
   if column_name in df.columns:
       column_index = df.columns.get_loc(column_name)
       column_indices[column_index] = value
print(column_indices)
 
 
# Select the sheet
sheet = wb['Sheet1']
 
column_names = [cell.value for cell in sheet[1]]
 
# Print the column names
print(column_names)
 
# Get the worksheet
ws = wb.active
 
#  Wrap the text in the selected cells and set the border
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.alignment = Alignment(vertical='top', horizontal='left', wrap_text=True) # Set alignment to top-left
        cell.border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
        ws.row_dimensions[cell.row].height = max(14, int(cell.font.size) + 2)
 
 
        # if len(str(cell.value)) > 20:  # Modify the threshold as needed
        #     cell.alignment = cell.alignment.copy(wrapText=True)
     
# Set the worksheet orientation to landscape
ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
 
# Color the header row with yellow background
header_fill = PatternFill(fill_type="solid", fgColor="FFFF00")
for cell in ws[1]:
    cell.fill = header_fill
 
 
# Save the Excel file
wb.save('output.xlsx')
 
 
workbook = Workbook("output.xlsx")
for sheet in workbook.getWorksheets():
   sheet.autoFitColumns()
   sheet.autoFitRows()
 
# Get the default worksheet
worksheet = workbook.getWorksheets().get(0)
 
# Set the page orientation to landscape
worksheet.getPageSetup().setOrientation(2)
 
# Set the paper size to A4
 
worksheet.getPageSetup().setPaperSize(9)
# Set the maximum rows per page
worksheet.getPageSetup().setFitToPagesTall(1)
worksheet.getPageSetup().setFitToPagesWide(0)
worksheet.setPageBreakPreview(False)
 
if len(column_indices) == 0: 
    column_indices = column_indices
    
    for index, width in column_indices.items():
        sheet.getCells().setColumnWidth(index, width)
    
    cells = sheet.getCells()
    for row in cells.getRows():
        for index in column_indices:
            cell = row.get(index)
            style = cell.getStyle()
            style.setTextWrapped(True)
            cell.setStyle(style)
       
# for sheet in workbook.getWorksheets():
#    sheet.autoFitColumns()
#    sheet.autoFitRows()
 
# Save as PDF
pdf_options = PdfSaveOptions()
pdf_options.setOnePagePerSheet(True)
# pdf_options.setAllColumnsInOnePagePerSheet(True)
workbook.save("output.pdf", pdf_options)

{'NOME DO SANCIONADO': 35, 'NOME INFORMADO PELO ÓRGÃO': 35, 'RAZÃO SOCIAL - CADASTRO RECEITA': 40, 'ÓRGÃO SANCIONADOR': 35, 'DETALHAMENTO': 35, 'NOME INFORMADO PELO ÓRGÃO SANCIONADOR': 35, 'PUBLICAÇÃO': 35}
{4: 35, 6: 40, 17: 35, 14: 35, 5: 35, 13: 35}
['CADASTRO', 'CÓDIGO DA SANÇÃO', 'TIPO DE PESSOA', 'CPF OU CNPJ DO SANCIONADO', 'NOME DO SANCIONADO', 'NOME INFORMADO PELO ÓRGÃO SANCIONADOR', 'RAZÃO SOCIAL - CADASTRO RECEITA', 'NOME FANTASIA - CADASTRO RECEITA', 'NÚMERO DO PROCESSO', 'CATEGORIA DA SANÇÃO', 'DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO', 'DATA PUBLICAÇÃO', 'PUBLICAÇÃO', 'DETALHAMENTO', 'DATA DO TRÂNSITO EM JULGADO', 'ABRAGÊNCIA DEFINIDA EM DECISÃO JUDICIAL', 'ÓRGÃO SANCIONADOR', 'UF ÓRGÃO SANCIONADOR', 'ESFERA ÓRGÃO SANCIONADOR', 'FUNDAMENTAÇÃO LEGAL']
